In [1]:
import pandas as pd

# Cargar los datos
df = pd.read_csv('data/interacciones.csv')

# Mostrar las primeras filas para tener una vista general
print(df.head())
print(df.info())

# Limpieza previa
# Eliminamos columnas innecesarias que no vamos a usar para el análisis
df.drop(columns=['IND_LAST', 'METADATA_FILENAME', 'METADATA_FILE_LAST_MODIFIED', 'META_TS', 'TS_CARGA', 'TS_PROCESSAMENT'], inplace=True)

# Verificar los valores nulos
print(df.isnull().sum())

# Rellenar los valores nulos con 0 (en este caso tiene sentido en las columnas numéricas)
df.fillna(0, inplace=True)

# Convertimos las columnas a los tipos correctos
df['USER_ID'] = df['USER_ID'].astype(int)
df['DISCUSSION_ENTRY_ID'] = df['DISCUSSION_ENTRY_ID'].astype(int)
df['RATING'] = df['RATING'].astype(int)
df['FORCED_READ_STATE'] = df['FORCED_READ_STATE'].astype(bool)

# Verificar los datos después de la limpieza
print(df.head())


         ID  USER_ID WORKFLOW_STATE FORCED_READ_STATE  DISCUSSION_ENTRY_ID  \
0  19033728   191041           read               NaN               890724   
1  19033727    97827           read               NaN               599902   
2  19033729   200825           read               NaN               639432   
3  19468980   166609           read               NaN               893052   
4  19468982   169487           read               NaN               798588   

   RATING                     META_TS                    TS_CARGA  \
0     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   
1     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   
2     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   
3     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   
4     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   

              TS_PROCESSAMENT  IND_LAST  \
0  2025-02-18 14:55:59.754000         0   
1  2025-02-18 14:55:59.754000 

In [2]:
import networkx as nx
import matplotlib.pyplot as plt

# Crear el grafo
G = nx.DiGraph()

# Añadir nodos y aristas
for _, row in df.iterrows():
    user_from = row['USER_ID']
    discussion_id = row['DISCUSSION_ENTRY_ID']
    weight = row['RATING']  # Peso de la arista es la calificación (RATING)
    
    # Añadir los nodos
    G.add_node(user_from)
    G.add_node(discussion_id)
    
    # Añadir la arista
    G.add_edge(user_from, discussion_id, weight=weight)

# Imprimir el número de nodos y aristas en el grafo
print(f"Nodos: {G.number_of_nodes()}, Aristas: {G.number_of_edges()}")


Nodos: 12203, Aristas: 10000


In [ ]:
# Graficar el grafo
plt.figure(figsize=(15, 15))
pos = nx.spring_layout(G, k=0.3)  # Establecer disposición de los nodos
nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue', font_size=5, edge_color='yellow')
plt.title("Grafo de Interacciones")
plt.show()


In [ ]:
# Grado de Centralidad
degree_centrality = nx.degree_centrality(G)
print("\nTop 10 usuarios por Grado de Centralidad:")
print(sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

# Centralidad de Intermediación
betweenness_centrality = nx.betweenness_centrality(G)
print("\nTop 10 usuarios por Intermediación:")
print(sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

# Centralidad de Cercanía
closeness_centrality = nx.closeness_centrality(G)
print("\nTop 10 usuarios por Cercanía:")
print(sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:10])


In [ ]:
import plotly.express as px
import pandas as pd
import plotly.io as pio
pio.renderers.default = 'browser'

# Crear un DataFrame con las métricas
centrality_df = pd.DataFrame({
    'User': list(degree_centrality.keys()),
    'DegreeCentrality': list(degree_centrality.values()),
    'Betweenness': list(betweenness_centrality.values()),
    'Closeness': list(closeness_centrality.values())
})

# Graficar el Top 10 de usuarios por Grado de Centralidad
fig = px.bar(centrality_df.sort_values(by='DegreeCentrality', ascending=False).head(10), 
             x='User', y='DegreeCentrality',
             title="Top 10 Usuarios por Grado de Centralidad",
             labels={'DegreeCentrality': 'Centralidad de Grado', 'User': 'Usuario'},
             color='DegreeCentrality')
fig.show()
